In [1]:
from google.colab import files
uploaded = files.upload()


Saving Israel and Palestine war.pdf to Israel and Palestine war.pdf


In [3]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [19]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.33.0
    Uninstalling openai-1.33.0:
      Successfully uninstalled openai-1.33.0


In [8]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 10.4 MB/s eta 0:00:00


In [6]:
import os
import PyPDF2
import openai
import faiss
import numpy as np
from google.colab import files



# Get the uploaded file name
pdf_file_path = list(uploaded.keys())[0]  # Use the first uploaded file

# Set up OpenAI API key
openai.api_key = 'OpenAI API key'

# Function to read content from a PDF file
def read_pdf(file_path):
    content = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        for page in range(num_pages):
            content += reader.pages[page].extract_text()
    return content

# Function to chunk text
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Function to get embeddings from OpenAI
def get_embeddings(texts):
    response = openai.Embedding.create(
        model="gpt-3.5-turbo",
        input=texts
    )
    return [embedding['embedding'] for embedding in response['data']]

# Initialize FAISS index
dimension = 1536  # Embedding dimension of 'text-embedding-ada-002'
index = faiss.IndexFlatL2(dimension)

# Read, chunk, and embed PDF content
all_chunks = []
all_titles = []

# Read the content of the PDF
content = read_pdf(pdf_file_path)
chunks = chunk_text(content)
embeddings = get_embeddings(chunks)

# Add embeddings to the index
index.add(np.array(embeddings).astype(np.float32))

all_chunks.extend(chunks)
all_titles.extend([pdf_file_path] * len(chunks))
print(f"Processed document: {pdf_file_path}")

# Function to search for similar embeddings
def search_embeddings(query, top_k=5):
    query_embedding = get_embeddings([query])[0]
    distances, indices = index.search(np.array([query_embedding]).astype(np.float32), top_k)
    return [(all_titles[idx], all_chunks[idx], distances[0][i]) for i, idx in enumerate(indices[0])]

# Function to generate prompts using context from the vector database
def generate_prompts_with_context(user_question, num_prompts=5, top_k=5):
    # Search for relevant context
    relevant_context = search_embeddings(user_question, top_k=top_k)
    combined_context = " ".join([context[1] for context in relevant_context])

    # Generate prompts using the combined context
    instruction = f"Generate {num_prompts} different prompts based on the following user question: '{user_question}' and the provided context."
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{instruction}\n\nContext:\n{combined_context}"}
        ],
        max_tokens=150,
        n=num_prompts,
        temperature=0.7
    )
    prompts = [choice['message']['content'].strip() for choice in response['choices']]
    return prompts

# Example usage
user_question = "Places to eat in Kenya"
generated_prompts = generate_prompts_with_context(user_question, num_prompts=5)
for i, prompt in enumerate(generated_prompts, start=1):
    print(f"Prompt {i}: {prompt}")


AuthenticationError: Incorrect API key provided: sk-proj-********************************************6WCu. You can find your API key at https://platform.openai.com/account/api-keys.